# Práctica 2 - Spark - Dataframes - SQL
---

#### 1. Cargue en un dataframe el archivo report.csv. Imprima en pantalla el esquema (columnas y tipos de datos asociados)

In [1]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home'

In [2]:
from pyspark.sql import SparkSession

In [3]:
from IPython.core.display import HTML

In [4]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [7]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
report = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('tps/report.csv')

In [10]:
report.printSchema()

root
 |-- report_year: integer (nullable = true)
 |-- agency_code: string (nullable = true)
 |-- agency_jurisdiction: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- violent_crimes: integer (nullable = true)
 |-- homicides: integer (nullable = true)
 |-- rapes: integer (nullable = true)
 |-- assaults: integer (nullable = true)
 |-- robberies: integer (nullable = true)
 |-- months_reported: integer (nullable = true)
 |-- crimes_percapita: double (nullable = true)
 |-- homicides_percapita: double (nullable = true)
 |-- rapes_percapita: double (nullable = true)
 |-- assaults_percapita: double (nullable = true)
 |-- robberies_percapita: double (nullable = true)



#### 2. Muestre en pantalla 3 registros del dataframe

In [11]:
report.show(3)

+-----------+-----------+-------------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|report_year|agency_code|agency_jurisdiction|population|violent_crimes|homicides|rapes|assaults|robberies|months_reported|crimes_percapita|homicides_percapita|rapes_percapita|assaults_percapita|robberies_percapita|
+-----------+-----------+-------------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|       1975|    NM00101|    Albuquerque, NM|    286238|          2383|       30|  181|    1353|      819|             12|          832.52|              10.48|          63.23|            472.68|             286.13|
|       1975|    TX22001|      Arlington, TX|    112478|           278|        5|   28|     132|      113|             12|          247.16| 

#### 3. ¿Qué datos contiene el archivo? ¿Qué información útil se podría obtener?

* ¿Está aumentando o disminuyendo la delincuencia en Estados Unidos?
* Delitos violentos en las jurisdicciones policiales

#### 4. Crear una "vista" llamada "crimes"

In [13]:
report.createOrReplaceTempView('crimes')

#### 5. a. Obtener la cantidad de crímenes violentos por año en Detroit, Michigan. Mostrar los resultados de los años recientes primero

In [43]:
consulta_5a = spark.sql("SELECT report_year, SUM(violent_crimes) FROM crimes WHERE agency_jurisdiction LIKE '%Detroit%' GROUP BY report_year")
consulta_5a.show()

+-----------+-------------------+
|report_year|sum(violent_crimes)|
+-----------+-------------------+
|       1990|              27747|
|       1975|              30387|
|       1977|              24070|
|       2003|              18724|
|       2007|              19690|
|       2015|              11846|
|       2006|              21391|
|       1978|              21602|
|       2013|              14501|
|       1988|              25805|
|       1997|              21976|
|       1994|              27471|
|       2014|              13622|
|       1979|              21021|
|       2004|              15913|
|       1991|              28262|
|       1982|              23746|
|       1989|              24956|
|       1996|              23239|
|       1998|              24427|
+-----------+-------------------+
only showing top 20 rows


#### 5. b. Mostrar la cantidad de homicidios en la ciudad de Las Vegas por año. Mostrar los resultados por año de forma creciente.

In [20]:
spark.sql("SELECT report_year, SUM(homicides) FROM crimes WHERE agency_jurisdiction LIKE '%Las Vegas%' GROUP BY report_year ORDER BY report_year").show()

+-----------+--------------+
|report_year|sum(homicides)|
+-----------+--------------+
|       1975|            37|
|       1976|            35|
|       1977|            59|
|       1978|            54|
|       1979|            75|
|       1980|            92|
|       1981|           106|
|       1982|            88|
|       1983|            74|
|       1984|            67|
|       1985|            57|
|       1986|            75|
|       1987|            47|
|       1988|            54|
|       1989|            67|
|       1990|            79|
|       1991|           103|
|       1992|            99|
|       1993|            91|
|       1994|           105|
+-----------+--------------+
only showing top 20 rows


#### 6. Realizar las consultas del ejercicios 5 utilizando los métodos del dataframe (select(), filter(), where(), etc)

In [41]:
report.filter(report.agency_jurisdiction.like('%Detroit%')).groupBy(report.report_year).sum('violent_crimes').show()

+-----------+-------------------+
|report_year|sum(violent_crimes)|
+-----------+-------------------+
|       1990|              27747|
|       1975|              30387|
|       1977|              24070|
|       2003|              18724|
|       2007|              19690|
|       2015|              11846|
|       2006|              21391|
|       1978|              21602|
|       2013|              14501|
|       1988|              25805|
|       1997|              21976|
|       1994|              27471|
|       2014|              13622|
|       1979|              21021|
|       2004|              15913|
|       1991|              28262|
|       1982|              23746|
|       1989|              24956|
|       1996|              23239|
|       1998|              24427|
+-----------+-------------------+
only showing top 20 rows


In [42]:
report.filter(report.agency_jurisdiction.like('%Las Vegas%')).groupBy(report.report_year).sum('homicides').orderBy(report.report_year.asc()).show()

+-----------+--------------+
|report_year|sum(homicides)|
+-----------+--------------+
|       1975|            37|
|       1976|            35|
|       1977|            59|
|       1978|            54|
|       1979|            75|
|       1980|            92|
|       1981|           106|
|       1982|            88|
|       1983|            74|
|       1984|            67|
|       1985|            57|
|       1986|            75|
|       1987|            47|
|       1988|            54|
|       1989|            67|
|       1990|            79|
|       1991|           103|
|       1992|            99|
|       1993|            91|
|       1994|           105|
+-----------+--------------+
only showing top 20 rows


#### 7. Guardar el resultado de la consulta 5.a en un único archivo JSON

In [45]:
consulta_5a.write.save('tps/salida', format = 'json')